# CONSULTAS

In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Conexión a PostgreSQL
user = "userGESTDB"
password = "passGESTDB"
host = "postgres_db"  # nombre del servicio en docker-compose
port = "5432"
db = "GESTDB"

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{db}")

# Leer una tabla completa
df = pd.read_sql("SELECT * FROM grado;", engine)
df.head()

,id,nombre,id_area,descripcion,salidas
0,1,Antropología Social y Cultural,1,"Conocer y apreciar la diversidad cultural, exp...","Graduados en diversidad cultural, patrimonio e..."
1,2,Antropología Social y Cultural,15,Conocer y apreciar la diversidad de sociedades...,"Graduados en diversidad cultural, patrimonio e..."
2,3,Bellas Artes,2,El Grado en Bellas Artes forma artistas plásti...,"Titulados en arte pueden ser pintores, esculto..."
3,4,Conservación y Restauración del Patrimonio Cul...,2,El Grado en Conservación y Restauración del Pa...,ConservadorRestaurador en instituciones públic...
4,5,Artes Escénicas,2,El Grado en Artes Escénicas capacita a jóvenes...,Profesionales con proyección multidisciplinar ...


In [2]:
# Ver grados con su rama, área y universidad

df_1 = pd.read_sql(
"""
SELECT  
    g.id AS id_grado, 
    g.nombre AS grado, 
    a.nombre AS area, 
    r.nombre AS rama, 
    u.nombre AS universidad 

FROM grado g 
JOIN area a ON g.id_area = a.id 
JOIN rama r ON a.id_rama = r.id 
JOIN grado_ofertado go ON g.id = go.id_grado 
JOIN facultad f ON go.id_facultad = f.id 
JOIN universidad u ON f.id_universidad = u.id 
ORDER BY r.nombre, g.nombre;
""", engine)
df_1.head()



,id_grado,grado,area,rama,universidad
0,52,Antropología Social y Cultural,Antropología Social y Cultural,Artes y Humanidades,Universidad Europea de Madrid
1,1,Antropología Social y Cultural,Antropología Social y Cultural,Artes y Humanidades,Universidad Autónoma de Madrid
2,1,Antropología Social y Cultural,Antropología Social y Cultural,Artes y Humanidades,Universidad Nacional de Educación a Distancia ...
3,1,Antropología Social y Cultural,Antropología Social y Cultural,Artes y Humanidades,Universidad Complutense de Madrid
4,32,Arqueología,Geografía e Historia,Artes y Humanidades,Universidad Nacional de Educación a Distancia ...


In [3]:
# Consulta de titulaciones posibles según la nota de admisión
df_2 = pd.read_sql(
    """
SELECT  
    g.nombre AS grado, 
    u.nombre AS universidad, 
    nc.nota AS nota_corte, 
    nc.año, 
    nc.convocatoria 
FROM nota_corte nc 
JOIN grado_ofertado go ON nc.id_grado_ofertado = go.id 
JOIN grado g ON go.id_grado = g.id 
JOIN facultad f ON go.id_facultad = f.id 
JOIN universidad u ON f.id_universidad = u.id 
WHERE nc.nota <= 11.0 

ORDER BY nc.nota DESC; 
    """, engine)
df_2.head()


,grado,universidad,nota_corte,año,convocatoria
0,Ingeniería en Tecnologías Industriales,Universidad Carlos III de Madrid,11.0,2025,ordinaria
1,Ingeniería en Tecnologías Industriales,Universidad a Distancia de Madrid UDIMA,11.0,2025,ordinaria
2,Ingeniería en Tecnologías Industriales,Universidad de Alcalá,11.0,2025,ordinaria
3,Ingeniería en Tecnologías Industriales,Universidad Politécnica de Madrid,11.0,2025,ordinaria
4,Ingeniería en Tecnologías Industriales,Universidad Pontificia Comillas ICAIICADE,11.0,2025,ordinaria


In [4]:
df.fillna({
        'nombre': 'Unknown', 
        'descripcion':'Este grado no contiene descripción',
        'salidas':'Este grado no contiene salidas'
        },
    inplace=True)
df

,id,nombre,id_area,descripcion,salidas
0,1,Antropología Social y Cultural,1,"Conocer y apreciar la diversidad cultural, exp...","Graduados en diversidad cultural, patrimonio e..."
1,2,Antropología Social y Cultural,15,Conocer y apreciar la diversidad de sociedades...,"Graduados en diversidad cultural, patrimonio e..."
2,3,Bellas Artes,2,El Grado en Bellas Artes forma artistas plásti...,"Titulados en arte pueden ser pintores, esculto..."
3,4,Conservación y Restauración del Patrimonio Cul...,2,El Grado en Conservación y Restauración del Pa...,ConservadorRestaurador en instituciones públic...
4,5,Artes Escénicas,2,El Grado en Artes Escénicas capacita a jóvenes...,Profesionales con proyección multidisciplinar ...
...,...,...,...,...,...
224,225,Ciencia de Datos e Inteligencia Artificial,75,El grado forma profesionales versátiles con só...,"Capacitados para gestionar Big Data, tomar dec..."
225,226,Inteligencia Artificial,75,Este Grado en Inteligencia Artificial proporci...,"Especialistas en tecnología, incluyendo seguri..."
226,227,Computación e Inteligencia Artificial,75,Este Grado proporciona una base sólida en info...,"Especialistas en tecnología digital, incluyend..."
227,228,Ingeniería Robótica Sotware,75,Formar profesionales que diseñen algoritmos y ...,La robótica se utiliza en industria y servicio...


In [5]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://elasticsearch:9200")

# Verifica la conexión
print(es.info().body)

{'name': 'b96cfab2bb10', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'eK64MsHJR0WNqUmQh325CA', 'version': {'number': '8.7.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '09520b59b6bc1057340b55750186466ea715e30e', 'build_date': '2023-03-27T16:31:09.816451435Z', 'build_snapshot': False, 'lucene_version': '9.5.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [6]:
# creamos los embedings de la descripcion de titulo
from sentence_transformers import SentenceTransformer

# Load a pre-trained Sentence Transformer model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Generate embeddings (vector representations) for descripciones
embeddings_descripcion = model.encode(df["descripcion"].values.tolist())

# Generate embeddings (vector representations) for salidas
embeddings_salidas = model.encode(df["salidas"].values.tolist())

/opt/conda/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [7]:
import json
from elasticsearch.helpers import bulk

index_name="informacion_grados"

def vector_bulk_index_data(es, data, index_name):
    batch_size = 50  # Reducir el tamaño del lote a 50
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        actions = []
        for idx, doc in enumerate(batch):
            doc["descripcion_vector"]=embeddings_descripcion[i + idx].tolist()
            doc["salidas_vector"]=embeddings_salidas[i + idx].tolist()
            actions.append({
                "_index": index_name,
                "_id": doc['id'],
                "_source": doc
            })
        # Capturar la respuesta para verificar errores
        resp = bulk(es, actions, raise_on_error=True)
        print("Indexed:", resp[0], "Errors:", resp[1])

descripciones = df.to_dict(orient='records')
vector_bulk_index_data(es, descripciones, index_name)

Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 29 Errors: []


### Queries sintácticas

In [8]:
import json

res = es.search(
    index=index_name,
    size=3,
    query={
        "term": {
            "nombre": "industrial"
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

Search Results:
Document ID: 190, Nombre Título: Ingeniería en Organización Industrial, Description: Un Ingeniero de Organización analiza y resuelve problemas para optimizar procesos empresariales, identificando prioridades estratégicas, recursos y beneficios. Su formación dual le permite tener una visión global de la empresa., Score: 4.1358886

Document ID: 192, Nombre Título: Ingeniería en Electrónica y Automática Industrial, Description: La titulación se centra en la tecnología punta y la creación de ingenieros mecánicos que mejoren producción y trabajo humano. Se aplican conocimientos a sistemas para mejorar procesos industriales, pero también se utilizan en medicina, agricultura, distribución de mercancías y energía., Score: 3.4236507

Document ID: 189, Nombre Título: Ingeniería en Diseño Industrial y Desarrollo de Producto, Description: La carrera de Diseño Industrial forma profesionales que diseñan productos innovadores, competitivos y eficientes desde el punto de vista funciona

In [9]:
res = es.search(
    index=index_name,
    size=3,
    query={
        "term": {
            "nombre": "informática"
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

Search Results:
Document ID: 214, Nombre Título: Ingeniería Informática, Description: La carrera de Ingeniería Informática forma profesionales que crean programas informáticos ajustados a las necesidades sociales y empresariales. Los ingenieros informáticos utilizan lenguajes de programación para analizar problemas cotidianos y desarrollar soluciones lógicas. La titulación se centra en la gestión y explotación de sistemas informáticos, incluyendo el diseño y desarrollo de software orientado al usuario final., Score: 5.2223105

Document ID: 215, Nombre Título: Ingeniería Informática Biomédica, Description: El grado en Ingeniería Informática Biomédica se enfoca en la aplicación de tecnología, informática y programación en salud y biomedicina., Score: 4.616037

Document ID: 220, Nombre Título: Matemáticas e Informática, Description: Este grado combina Matemáticas e Informática, enfatizando la interrelación entre ambas. Forma graduados con conocimientos generales en ambos campos para acced

### Queries léxcias

In [10]:
res = es.search(
    index="informacion_grados",
    size=3,
    query={
        "match": {
            "descripcion": "computación"
        }
    }
)


print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

Search Results:
Document ID: 203, Nombre Título: Matemática Computacional, Description: El Grado en Matemática Aplicada y Computación forma profesionales que aplican matemáticas en áreas como la ciencia, ingeniería y economía. Los estudiantes aprenden algoritmos, métodos numéricos y programación para resolver problemas reales., Score: 4.7130547

Document ID: 225, Nombre Título: Ciencia de Datos e Inteligencia Artificial, Description: El grado forma profesionales versátiles con sólida base en Matemáticas, Estadística y Computación. Los egresados pueden aplicar habilidades en tecnologías de la información, ciencia de datos e inteligencia artificial para diseñar estrategias de gestión de datos y sistemas de información. También deben considerar aspectos éticos, legales y sostenibles en el uso de técnicas de IA y datos., Score: 3.658999



In [11]:
res = es.search(
    index="informacion_grados",
    size=3,
    query={
        "bool": {
            "must": [
                {"match": {"descripcion": "comunicación"}},
                {"match": {"salidas": "medios audiovisuales"}}
            ],
            "must_not": [
                {"match": {"descripcion": "ingeniería"}},
                {"match": {"descripcion": "matemáticas"}}
            ]
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

Search Results:
Document ID: 126, Nombre Título: Comunicación y Medios Digitales, Description: El Grado en Periodismo forma a universitarios para realizar la función informativa con rigor, profundidad e idoneidad. Los estudiantes aprenden a transmitir noticias, redactar correctamente y expresarse en el lenguaje adecuado dependiendo del tipo de noticia y medio de comunicación. El objetivo es formar periodistas capacitados para trabajar en empresas relacionadas con la información y la comunicación., Score: 5.8352966

Document ID: 123, Nombre Título: Diseño Digital y Multimedia, Description: El grado forma a diseñadores altamente cualificados en tecnología, comunicación y arte. Los estudiantes deben ser creativos, permeables al cambio y tener deseos de aprender permanentemente., Score: 5.782515

Document ID: 16, Nombre Título: Diseño, Description: El Grado en Diseño combina teoría y práctica con otras disciplinas como Arte, Arquitectura y Comunicación. Ofrece cuatro menciones: Diseño gráf

In [12]:
res = es.search(
    index="informacion_grados",
    size=1,
    query={
        "bool": {
            "must": [
                {"match": {"descripcion": "gestión"}},
                {"match": {"salidas": "administración de empresas"}}
            ],
            "must_not": [
                {"match": {"descripcion": "finanzas"}},
                {"match": {"salidas": "contabilidad"}}
            ]
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

Search Results:
Document ID: 56, Nombre Título: Gestión Deportiva, Description: El Grado en Gestión Deportiva forma a profesionales en turismo deportivo, marketing deportivo, gestión de centros deportivos y asesoría de instalaciones deportivas., Score: 6.2876196



In [13]:
res = es.search(
    index="informacion_grados",
    size=3,
    query={
        "bool": {
            "must": [
                {"match": {"descripcion": "comportamiento humano"}},
                {"match": {"salidas": "social"}}
            ],
            "must_not": [
                {"term": {"nombre.keyword": "Psicología"}},
                {"term": {"nombre.keyword": "Biología Sanitaria"}}
            ]
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
    print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
    print("")

Search Results:
Document ID: 91, Nombre Título: Comportamiento y Ciencias Sociales, Description: Este programa multidisciplinario aborda la comprensión del comportamiento humano desde diferentes ámbitos sociales. No es un Grado de Psicología, sino una formación holística que combina Ciencias Sociales para analizar y influir en el comportamiento humano. Se enfoca en desarrollar profesionales capaces de ayudar a empresas y organizaciones a ser más competitivas y centradas en el cliente., Score: 12.187498

Document ID: 61, Nombre Título: Ciencias Políticas, Description: El grado en Ciencias Políticas ofrece una formación pluridisciplinar centrada en el análisis de los procesos políticos contemporáneos. Se incluyen conocimientos teóricos y prácticos sobre comunicación política, comportamiento electoral, instituciones y políticas públicas, entre otros., Score: 6.8571663



In [14]:
res = es.search(
    index="informacion_grados",
    size=2,
    query={
        "bool": {
            "must": [
                {"match": {"descripcion": "arte digital"}},
                {"match": {"salidas": "diseño"}}
            ],
            "must_not": [
                {"match": {"descripcion": "docencia"}},
                {"match": {"descripcion": "ingeniería"}}
            ]
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
    print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
    print("")


Search Results:
Document ID: 128, Nombre Título: Creación y Narración de Videojuegos, Description: La titulación forma profesionales con pensamiento crítico y capacidad de expresión, cubriendo contenidos humanísticos, lenguaje, diseño narrativo, dirección creativa, arte digital, programación y emprendimiento. Se enfoca en el desarrollo de videojuegos, pero también prepara para la informática y el diseño en general., Score: 7.942289

Document ID: 20, Nombre Título: Diseño Gráfico y Multimedia, Description: La carrera de Diseño Multimedia y Gráfico ofrece una preparación multidisciplinar con un enfoque técnico y práctico. Combina habilidades técnicas con conocimientos transversales como creatividad, estética e historia del arte., Score: 6.527178



### Queries semánticas

In [15]:
query_sentence = "análisis de datos"
query_vector = model.encode([query_sentence])[0]

parameters = {
     "field":"descripcion_vector",
     "query_vector": query_vector,
     "k":5,
     "num_candidates":100
}
res = es.search(
    index=index_name, 
    knn=parameters)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

BadRequestError: BadRequestError(400, 'search_phase_execution_exception', 'failed to create query: [knn] queries are only supported on [dense_vector] fields')